In [34]:
import time
import random
import logging
from typing import List, Dict
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# -------------------------
# Logging Configuration
# -------------------------
logging.basicConfig(
    level=logging.INFO,
    format='%(message)s'
)

# -------------------------
# Feature Extractor
# -------------------------
def extract_features(deployment: Dict) -> List[float]:
    return [
        deployment['files_changed'],
        deployment['test_coverage'],
        len(deployment['services_affected']),
        deployment['author_risk_score'],
        deployment['change_complexity'],
    ]

# -------------------------
# Risk Predictor
# -------------------------
class RiskPredictor:
    def __init__(self):
        self.model = Pipeline([
            ('scaler', StandardScaler()),
            ('clf', RandomForestClassifier(n_estimators=100, random_state=42))
        ])
        self.is_trained = False
        self.df = None

    def train(self, csv_path: str):
        self.df = pd.read_csv(csv_path)
        X = self.df[['files_changed', 'test_coverage', 'services_affected',
                     'author_risk_score', 'change_complexity']].values
        y = self.df['rollback'].values
        self.model.fit(X, y)
        self.is_trained = True
        print("\n✅ Model trained on deployment dataset.\n")

    def predict(self, features: List[float]) -> float:
        if not self.is_trained:
            raise RuntimeError("❗ Model not trained.")
        return self.model.predict_proba([features])[0][1]

# -------------------------
# Monitoring & Anomaly Detection
# -------------------------
def monitor_metrics() -> Dict[str, float]:
    return {
        'error_rate': round(random.uniform(0.0, 0.2), 3),
        'latency_p95': round(random.uniform(100, 500), 2),
        'cpu_usage': round(random.uniform(0.4, 0.95), 2),
    }

def detect_anomaly(metrics: Dict[str, float]) -> bool:
    detected = False
    if metrics['error_rate'] > 0.1:
        print("❗ Anomaly: High error rate")
        detected = True
    if metrics['latency_p95'] > 300:
        print("❗ Anomaly: High latency")
        detected = True
    if metrics['cpu_usage'] > 0.85:
        print("❗ Anomaly: High CPU usage")
        detected = True
    return detected

def trigger_rollback():
    print("🚨 Rollback triggered due to high risk or anomaly!\n")

# -------------------------
# Display Inputs and Prediction
# -------------------------
def show_user_inputs_and_prediction(features: List[float], risk_score: float):
    feature_names = ['files_changed', 'test_coverage', 'services_affected',
                     'author_risk_score', 'change_complexity']
    print("\n🔎 Raw Input Features:")
    for name, value in zip(feature_names, features):
        print(f" - {name}: {value}")
    print(f"\n🔍 Predicted rollback probability: {risk_score:.2f}")

# -------------------------
# Deployment Execution
# -------------------------
def deploy(deployment: Dict, predictor: RiskPredictor, label: str):
    print(f"\n🚀 Starting {label} deployment analysis...")
    features = extract_features(deployment)
    risk_score = predictor.predict(features)
    show_user_inputs_and_prediction(features, risk_score)

    if risk_score > 0.6:
        print(f"⚠️ {label}: Deployment stopped due to high predicted rollback risk.\n")
        return

    print(f"🛠️ {label}: Deployment proceeding...")
    for i in range(3):
        metrics = monitor_metrics()
        print(f"📊 {label} Metrics Check {i+1}: {metrics}")
        if detect_anomaly(metrics):
            trigger_rollback()
            return
        time.sleep(1)

    print(f"✅ {label} Deployment completed successfully.\n")

# -------------------------
# Train & Simulate
# -------------------------
if __name__ == "__main__":
    predictor = RiskPredictor()
    predictor.train("deployment_dataset_5000.csv")  # Upload this file to Colab

    # Pick random high-risk and low-risk deployments from dataset
    high_risk_sample = predictor.df[predictor.df['rollback'] == 1].sample(1).iloc[0]
    low_risk_sample = predictor.df[predictor.df['rollback'] == 0].sample(1).iloc[0]

    high_risk_deployment = {
        'files_changed': high_risk_sample['files_changed'],
        'test_coverage': high_risk_sample['test_coverage'],
        'services_affected': ['auth', 'checkout', 'payment'],
        'author_risk_score': high_risk_sample['author_risk_score'],
        'change_complexity': high_risk_sample['change_complexity']
    }

    low_risk_deployment = {
        'files_changed': low_risk_sample['files_changed'],
        'test_coverage': low_risk_sample['test_coverage'],
        'services_affected': ['auth'],
        'author_risk_score': low_risk_sample['author_risk_score'],
        'change_complexity': low_risk_sample['change_complexity']
    }

    deploy(high_risk_deployment, predictor, "High-Risk")
    deploy(low_risk_deployment, predictor, "Low-Risk")

    # -------------------------
    # User Input Deployment
    # -------------------------
    print("\n📥 Enter your own deployment details below:")
    try:
        user_input = {
            'files_changed': int(input("Files changed: ")),
            'test_coverage': float(input("Test coverage (%): ")),
            'services_affected': input("Comma-separated services affected (e.g. auth,payment): ").split(','),
            'author_risk_score': float(input("Author risk score (0-1): ")),
            'change_complexity': float(input("Change complexity (1-10): ")),
        }
        deploy(user_input, predictor, "User-Defined")
    except Exception as e:
        print(f"❌ Invalid input: {e}")



✅ Model trained on deployment dataset.


🚀 Starting High-Risk deployment analysis...

🔎 Raw Input Features:
 - files_changed: 34.0
 - test_coverage: 60.86
 - services_affected: 3
 - author_risk_score: 0.24
 - change_complexity: 5.5

🔍 Predicted rollback probability: 1.00
⚠️ High-Risk: Deployment stopped due to high predicted rollback risk.


🚀 Starting Low-Risk deployment analysis...

🔎 Raw Input Features:
 - files_changed: 10.0
 - test_coverage: 71.94
 - services_affected: 1
 - author_risk_score: 0.44
 - change_complexity: 2.4

🔍 Predicted rollback probability: 0.01
🛠️ Low-Risk: Deployment proceeding...
📊 Low-Risk Metrics Check 1: {'error_rate': 0.072, 'latency_p95': 117.77, 'cpu_usage': 0.41}
📊 Low-Risk Metrics Check 2: {'error_rate': 0.173, 'latency_p95': 372.33, 'cpu_usage': 0.74}
❗ Anomaly: High error rate
❗ Anomaly: High latency
🚨 Rollback triggered due to high risk or anomaly!


📥 Enter your own deployment details below:
Files changed: 20
Test coverage (%): 20
Comma-separated s